In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from cmdstanpy import CmdStanModel
from baynes.fplot import FitPlotter
from baynes.model_utils import get_stan_file
import pandas as pd
from scipy import stats
import arviz as az
sns.set_style('ticks')
#sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.6)
plt.rc("axes.spines", top=False, right=False)

In [ ]:
fplot = FitPlotter(output_dir="/home/pietro/work/TESI/thesis/figures/ch2/poisson/", output_format='.pdf')

In [ ]:
stan_file = get_stan_file('funnel.stan')
model = CmdStanModel(stan_file=stan_file,
                     cpp_options={'STAN_THREADS': True, 'jN': 4})
print(model.code())
fit = model.sample({},
                   chains=4,
                   iter_warmup=1000,
                   iter_sampling=1000,
                   save_warmup=True,
                   show_progress=True,
                   adapt_delta=0.9)
print(fit.diagnose())

In [ ]:
xmax=20
ymin, ymax = -2, 5
X = np.linspace(-xmax, xmax, 1001)
Y= np.linspace(ymin, ymax, 1001)
Z=[]
for y in Y:
    Z.append(stats.norm.pdf(X, 0, np.exp(y))* stats.norm.pdf(y, loc=0, scale=3))
Z = np.array(Z)
levels=Z[[900, 850, 800, 750, 700, 650, 600], 500]
ax = fplot.new_figure('funnel').subplots()
ax.contour(X, Y, Z, levels=levels, colors='grey', alpha=0.5, zorder=0)
sns.scatterplot(fit.draws_pd(), x='x', y='y', hue='divergent__', legend=False, ax=ax)
ax.set_xlim(-xmax, xmax)
ax.set_ylim(ymin,ymax)